## Setup the Simulator

In [25]:
%load_ext autoreload
%autoreload 2


import os
import json
import numpy as np
import sys
sys.path.append('..')
sys.path.append('../simulation')
from dataset_utils import execute_script_utils as utils
from evolving_graph import scripts
from GraphReader import GraphReader, remove_nodes_from_graph, reference_graph_file, base_graph_file, init_graph_file, scene_num

print(f'Using scene {int(scene_num)-1}, i.e. TrimmedTestScene{scene_num}')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using scene 1, i.e. TrimmedTestScene2


## Add necessary objects

Some objects need to be removed.

In [26]:
nodes = ['phone', 'cellphone', 'dresser', 'keyboard', 'orchid', 'pillow']
remove_nodes_from_graph(reference_graph_file, nodes, base_graph_file)

In [27]:
base_graph = GraphReader(base_graph_file)

Fill in objects placements in `AddObjects.py` with statements of the form :
 ```
 base_graph.add("food_apple", "ON", ref_nodes['dining_room']['kitchen_counter'])
 ```

### Some 'furniture' was missing in the graphs so we add them first

In [28]:
ref_nodes = base_graph.usable_nodes_by_room

washing_machine_id = base_graph.new_obj_id
base_graph.add("washing_machine", "INSIDE", ref_nodes['dining_room']['dining_room'], category="Appliances")
base_graph.usable_nodes_by_room['dining_room']['washing_machine'] = washing_machine_id
base_graph.new_obj_id += 1

stove_id = base_graph.new_obj_id
base_graph.add("stove", "ON", ref_nodes['dining_room']['kitchen_counter'], category="Appliances")
base_graph.usable_nodes_by_room['dining_room']['stove'] = stove_id
base_graph.new_obj_id += 1

cabinet_id = base_graph.new_obj_id
base_graph.add("kitchen_cabinet", "INSIDE", ref_nodes['dining_room']['dining_room'], category="Furniture")
base_graph.usable_nodes_by_room['dining_room']['kitchen_cabinet'] = cabinet_id
base_graph.new_obj_id += 1

fridge_id = base_graph.new_obj_id
base_graph.add("fridge", "INSIDE", ref_nodes['dining_room']['dining_room'], category="Appliances")
base_graph.usable_nodes_by_room['dining_room']['fridge'] = fridge_id
base_graph.new_obj_id += 1

dresser_id = base_graph.new_obj_id
base_graph.add("dresser", "INSIDE", ref_nodes['bedroom']['bedroom'], category="Furniture")
base_graph.usable_nodes_by_room['bedroom']['dresser'] = dresser_id
base_graph.new_obj_id += 1

shoe_rack_id = base_graph.new_obj_id
base_graph.add("shoe_rack", "INSIDE", ref_nodes['dining_room']['dining_room'], category="Furniture")
base_graph.usable_nodes_by_room['dining_room']['shoe_rack'] = shoe_rack_id
base_graph.new_obj_id += 1


base_graph.node_map['<washing_machine>'] = f'<washing_machine> ({washing_machine_id})'
base_graph.node_map['<stove>'] = f'<stove> ({stove_id})'
base_graph.node_map['<kitchen_cabinet>'] = f'<kitchen_cabinet> ({cabinet_id})'
base_graph.node_map['<fridge>'] = f'<fridge> ({fridge_id})'
base_graph.node_map['<dresser>'] = f'<dresser> ({dresser_id})'
base_graph.node_map['<shoe_rack>'] = f'<shoe_rack> ({shoe_rack_id})'

In [29]:
print('List of available locations :')
for room,nodelist in ref_nodes.items():
    print('##### ',room,' #####')
    furniture = list(nodelist.keys())
    furniture.sort()
    print(furniture)

List of available locations :
#####  bathroom  #####
['bathroom', 'bathroom_cabinet', 'bathroom_counter', 'ceilinglamp', 'curtain', 'door', 'drawing', 'faucet', 'light', 'mat', 'shower', 'sink', 'toilet', 'walllamp']
#####  dining_room  #####
['bench', 'bookshelf', 'ceilinglamp', 'coffe_maker', 'cupboard', 'dining_room', 'dishwasher', 'door', 'drawing', 'dresser', 'faucet', 'freezer', 'fridge', 'kitchen_cabinet', 'kitchen_counter', 'knifeblock', 'light', 'mat', 'microwave', 'oven', 'pot', 'powersocket', 'shoe_rack', 'sink', 'stove', 'stovefan', 'table', 'toaster', 'trashcan', 'tray', 'tvstand', 'walllamp', 'washing_machine']
#####  bedroom  #####
['bed', 'bedroom', 'bookshelf', 'ceilinglamp', 'chair', 'computer', 'cpuscreen', 'curtain', 'desk', 'door', 'drawing', 'dresser', 'filing_cabinet', 'light', 'mat', 'mouse', 'mousepad', 'nightstand', 'photoframe', 'sofa', 'table', 'tablelamp']
#####  home_office  #####
['bookshelf', 'ceilinglamp', 'chair', 'computer', 'cpuscreen', 'curtain', 'd

In [30]:
%run -i 'AddObjects.py'

States not found for bleach
States not found for bottlebrush
States not found for dishtowel
States not found for disinfectant_wipes
States not found for dustbrush
States not found for featherduster
States not found for kitchentowel
States not found for magiceraser
States not found for multicleaner
States not found for recyclebin
States not found for scouringpad
States not found for scraper
States not found for scrubber
States not found for steelwool
States not found for trashbag
States not found for washcloth
States not found for whiskbroom


In [31]:
base_graph.write(init_graph_file)

In [32]:
with open('ActivityObjects.json') as f:
    activities = list(json.load(f).keys())

for act in activities:
    activity_dir = os.path.join('data/sourcedScriptsByActivity',act)
    if not os.path.exists(activity_dir):
        os.makedirs(activity_dir)